In [1]:
import numpy as np
import healpy as hp
import h5py
from astropy.io import fits


from utils.calc_volume_numdens_scaled_richness import (
    make_astropy_flamingo_cosmo,
    f_sky_from_area_deg2,
    volume_simple_astropy_comoving_volume,
    volume_astropy_distance_shell,
)
from utils.calc_effective_area_DESY3 import(
    solve_M_from_NK,
    area_vs_z_bins_DES_weighted,
    shell_volume_per_deg2_astropy,
    volume_weighted_area_and_volume,
)
cosmo = make_astropy_flamingo_cosmo()

# DES Y3 data

## (1) Load data and cluster counts in richness bins

In [2]:
DESY3_loc = "/projects/lyang4/projection_effects/cluster_cosmology/DESY3/"

fname = DESY3_loc+"y3_redmapper_v6.4.22+2_release.h5"
with h5py.File(fname, "r") as f_desy3:
    # Load full random catalog
    rand_data = f_desy3.require_group('randoms')
    rand_ra         = rand_data['ra'][:]
    rand_dec        = rand_data['dec'][:]
    rand_weight     = rand_data['weight'][:]
    rand_ztrue      = rand_data['ztrue'][:]
    rand_lam_in     = rand_data['lambda_in'][:]
    rand_ave_lamout = rand_data['avg_lambdaout'][:]

    ### load index of random clusters for cosmology analysis 
    rs = f_desy3["index/redmapper/lgt20/random_select"][...]
    
    # Load full cluster catalog
    cluster_cats = f_desy3["catalog/cluster"]
    z_lam = cluster_cats["z_lambda"][:]
    lam   = cluster_cats["lambda_chisq"][:]
    
    # Load index of clusters used for cosmology analyses
    idx = f_desy3["index/redmapper/lgt20/select"][:]

### Random for cosmo
mask_cosmo = np.zeros(rand_ra.size, dtype=bool)
mask_cosmo[rs] = True

### Cluster counts in four richness bins from full DES Y3 cluster catalog
sel_z = (z_lam >= 0.2) & (z_lam <= 0.35)
lam_sel  = lam[sel_z]

lam_bins = [20, 30, 45, 60, np.inf]

for lo, hi in zip(lam_bins[:-1], lam_bins[1:]):
    n = np.sum((lam_sel >= lo) & (lam_sel < hi))
    print(lo, hi, "clu_number (full)", n)

print(60*'-')
### Cluster counts in four richness bins from full DES Y3 cluster catalog
sel_cosmo = np.zeros(z_lam.size, dtype=bool)
sel_cosmo[idx] = True

sel_z_cosmo = (sel_cosmo & sel_z)
lam_sel_cosmo   = lam[sel_z_cosmo]

for lo, hi in zip(lam_bins[:-1], lam_bins[1:]):
    n = np.sum((lam_sel_cosmo >= lo) & (lam_sel_cosmo < hi))
    print(lo, hi, "clu_number (cosmo)", n)

20 30 clu_number (full) 2416
30 45 clu_number (full) 1112
45 60 clu_number (full) 373
60 inf clu_number (full) 259
------------------------------------------------------------
20 30 clu_number (cosmo) 2186
30 45 clu_number (cosmo) 1019
45 60 clu_number (cosmo) 336
60 inf clu_number (cosmo) 238


## (2) Calculate effetive area $A_{\rm eff}$ --- convergence test 
###   --- Three cases: 1) full cluster catalog; 2) clusters for cosmology analysis; 3) richenss-selected clusters
###   --- Three "$\tt nside$" choices


In [3]:
z_edges = np.arange(0.2,0.65+0.05, 0.05)

print(30*'+', 'All random clusters', 30*'+')
print(120*' ')
for ns in [256, 512, 1024]:
    # 1) No selection beyond z-bin
    out = area_vs_z_bins_DES_weighted(rand_ra, rand_dec, rand_ztrue, rand_weight,
                                      z_edges, nside=ns, min_points=2000, show_empty_fraction=False)
    
    # print("nside", ns, "A_eff", out["A_eff_deg2"], "mean", np.mean(out["A_eff_deg2"]))

    vw = volume_weighted_area_and_volume(z_edges, out["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    # print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])
    print(80*'-')


print(120*' ')
print(30*'+', 'Only random clusters for cosmology', 30*'+')
print(120*' ')
for ns in [256, 512, 1024]:
    # 2) No selection beyond z-bin
    out = area_vs_z_bins_DES_weighted(
        rand_ra, rand_dec, rand_ztrue, rand_weight,
        z_edges, nside=ns, min_points=2000,
        base_mask=mask_cosmo,
        show_empty_fraction=False,
    )
    # print("nside", ns, "A_eff", out["A_eff_deg2"], "mean", np.mean(out["A_eff_deg2"]))
    

    vw = volume_weighted_area_and_volume(z_edges, out["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    # print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])
    print(80*'-')


print(120*' ')
print(30*'+', 'All random clusters with richness selection', 30*'+')
print(120*' ')
for ns in [256, 512, 1024]:
    # 3) Observed richness selection via avg_lambdaout, e.g., [20,30)
    sel_20_30 = (rand_ave_lamout >= 20) & (rand_ave_lamout < 30)
    out_20_30 = area_vs_z_bins_DES_weighted(rand_ra, rand_dec, rand_ztrue, rand_weight,
                                            z_edges, nside=ns, select_mask=sel_20_30, min_points=2000,
                                            show_empty_fraction=False
                                           )
    
    # print("nside", ns, "A_eff_sel", out_20_30["A_eff_deg2"], "mean", np.mean(out_20_30["A_eff_deg2"]))
    print(80*'-')

    vw = volume_weighted_area_and_volume(z_edges, out_20_30["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    # print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])

++++++++++++++++++++++++++++++ All random clusters ++++++++++++++++++++++++++++++
                                                                                                                        
nside 256 A_volw (deg^2): 4672.655845704908
--------------------------------------------------------------------------------
nside 512 A_volw (deg^2): 4476.982640675091
--------------------------------------------------------------------------------
nside 1024 A_volw (deg^2): 4356.41159510808
--------------------------------------------------------------------------------
                                                                                                                        
++++++++++++++++++++++++++++++ Only random clusters for cosmology ++++++++++++++++++++++++++++++
                                                                                                                        
nside 256 A_volw (deg^2): 4592.440705268808
---------------------------------------

## (2) Calculate effetive area $A_{\rm eff}$ --- clusters with $z\in [0.2, 0.35]$ 

In [4]:
z_edges = np.arange(0.2,0.35+0.05, 0.05)

print(30*'+', 'All random clusters', 30*'+')
print(120*' ')
for ns in [256, 512, 1024]:
    # 1) No selection beyond z-bin
    out = area_vs_z_bins_DES_weighted(
        rand_ra, rand_dec, rand_ztrue, rand_weight,
        z_edges, nside=ns, min_points=2000, 
        show_empty_fraction=True
    )
    
    # print("nside", ns, "A_eff", out["A_eff_deg2"], "mean", np.mean(out["A_eff_deg2"]))

    vw = volume_weighted_area_and_volume(z_edges, out["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])
    print(80*'-')


print(120*' ')
print(30*'+', 'Only random clusters for cosmology', 30*'+')
print(120*' ')
for ns in [256, 512, 1024]:
    # 2) No selection beyond z-bin
    out = area_vs_z_bins_DES_weighted(
        rand_ra, rand_dec, rand_ztrue, rand_weight,
        z_edges, nside=ns, min_points=2000,
        base_mask=mask_cosmo,
        show_empty_fraction=True,
    )
    # print("nside", ns, "A_eff", out["A_eff_deg2"], "mean", np.mean(out["A_eff_deg2"]))
    

    vw = volume_weighted_area_and_volume(z_edges, out["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])
    print(80*'-')


print(120*' ')
print(30*'+', 'All random clusters with richness selection', 30*'+')
print(120*' ')
for ns in [256, 512, 1024]:
    # 3) Observed richness selection via avg_lambdaout, e.g., [20,30)
    sel_20_30 = (rand_ave_lamout >= 20) & (rand_ave_lamout < 30)
    out_20_30 = area_vs_z_bins_DES_weighted(
        rand_ra, rand_dec, rand_ztrue, rand_weight,
        z_edges, nside=ns, select_mask=sel_20_30, min_points=2000,
        show_empty_fraction=True
        )
    
    # print("nside", ns, "A_eff_sel", out_20_30["A_eff_deg2"], "mean", np.mean(out_20_30["A_eff_deg2"]))
    print(80*'-')

    vw = volume_weighted_area_and_volume(z_edges, out_20_30["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])

++++++++++++++++++++++++++++++ All random clusters ++++++++++++++++++++++++++++++
                                                                                                                        
N/Mhat = 2.3360730816189634 empty fraction ~ 0.09670665269604271 for redshift bin: [0.2, 0.25)
N/Mhat = 2.765347426869546 empty fraction ~ 0.06295422355930082 for redshift bin: [0.25, 0.3)
N/Mhat = 2.872219221232192 empty fraction ~ 0.05657323863440083 for redshift bin: [0.3, 0.35)
nside 256 A_volw (deg^2): 4643.926103872903
V_eff_total ((Mpc/h)^3): 331116308.1352404
--------------------------------------------------------------------------------
N/Mhat = 0.6031236344378713 empty fraction ~ 0.5471000237848915 for redshift bin: [0.2, 0.25)
N/Mhat = 0.7180737953571432 empty fraction ~ 0.48769074398514856 for redshift bin: [0.25, 0.3)
N/Mhat = 0.7451306107414766 empty fraction ~ 0.4746722985901022 for redshift bin: [0.3, 0.35)
nside 512 A_volw (deg^2): 4478.821323142251
V_eff_total ((Mpc/h

### DES Y3: $A_{\rm eff}$ and $V_{\rm eff}$ used for FLAMINGO abundance matching 

In [5]:
z_edges = np.arange(0.2,0.35+0.05, 0.05)

print(30*'+', 'All random clusters', 30*'+')
print(120*' ')
for ns in [512]:
    # 1) No selection beyond z-bin
    out = area_vs_z_bins_DES_weighted(
        rand_ra, rand_dec, rand_ztrue, rand_weight,
        z_edges, nside=ns, min_points=2000, 
        show_empty_fraction=True
    )
    
    # print("nside", ns, "A_eff", out["A_eff_deg2"], "mean", np.mean(out["A_eff_deg2"]))

    vw = volume_weighted_area_and_volume(z_edges, out["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])
    print(80*'-')

z_edges = np.arange(0.2,0.35+0.03, 0.03)

print(30*'+', 'All random clusters', 30*'+')
print(120*' ')
for ns in [512]:
    # 1) No selection beyond z-bin
    out = area_vs_z_bins_DES_weighted(
        rand_ra, rand_dec, rand_ztrue, rand_weight,
        z_edges, nside=ns, min_points=2000, 
        show_empty_fraction=True
    )
    
    # print("nside", ns, "A_eff", out["A_eff_deg2"], "mean", np.mean(out["A_eff_deg2"]))

    vw = volume_weighted_area_and_volume(z_edges, out["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])
    print(80*'-')

z_edges = np.arange(0.2,0.35+0.01, 0.01)

print(30*'+', 'All random clusters', 30*'+')
print(120*' ')
for ns in [512]:
    # 1) No selection beyond z-bin
    out = area_vs_z_bins_DES_weighted(
        rand_ra, rand_dec, rand_ztrue, rand_weight,
        z_edges, nside=ns, min_points=2000, 
        show_empty_fraction=True
    )
    
    # print("nside", ns, "A_eff", out["A_eff_deg2"], "mean", np.mean(out["A_eff_deg2"]))

    vw = volume_weighted_area_and_volume(z_edges, out["A_eff_deg2"], cosmo)
    print("nside", ns, "A_volw (deg^2):", vw["A_volw_deg2"])
    print("V_eff_total ((Mpc/h)^3):", vw["V_eff_total"])
    # print("V_eff_bins:", vw["V_eff_bins"])
    print(80*'-')

++++++++++++++++++++++++++++++ All random clusters ++++++++++++++++++++++++++++++
                                                                                                                        
N/Mhat = 0.6031236344378713 empty fraction ~ 0.5471000237848915 for redshift bin: [0.2, 0.25)
N/Mhat = 0.7180737953571432 empty fraction ~ 0.48769074398514856 for redshift bin: [0.25, 0.3)
N/Mhat = 0.7451306107414766 empty fraction ~ 0.4746722985901022 for redshift bin: [0.3, 0.35)
nside 512 A_volw (deg^2): 4478.821323142251
V_eff_total ((Mpc/h)^3): 319344181.6568669
--------------------------------------------------------------------------------
++++++++++++++++++++++++++++++ All random clusters ++++++++++++++++++++++++++++++
                                                                                                                        
N/Mhat = 0.3282888018865052 empty fraction ~ 0.7201550075448399 for redshift bin: [0.2, 0.23)
N/Mhat = 0.38544884054452994 empty fraction ~ 0.6

# DES Y1

## (1) Load data 

In [6]:
# file in the Y1A1 public directory:
# redmapper_y1a1_public_v6.4_zmask.fits.gz  (see DES listing)
# DESY3_loc = "/projects/lyang4/projection_effects/cluster_cosmology/DESY3/"


fname = DESY3_loc+'des_y1/redmapper_y1a1_public_v6.4_catalog.fits.gz'
with fits.open(fname) as hdul:
    hdul.verify('fix')
    hdata_cluster = hdul[1].data

# cols = hdul[1].columns
# print(cols)

lam_desy1   = hdata_cluster['LAMBDA']
zlam_desy1  = hdata_cluster['Z_LAMBDA']
# zspec_desy1 = hdata_cluster['Z_SPEC']
print(lam_desy1.min(), lam_desy1.max(), lam_desy1.size)

lam_sel = lam_desy1[(zlam_desy1>=0.2)&(zlam_desy1<=0.35)]
lam_bins = [20, 30, 45, 60, np.inf]
for lo, hi in zip(lam_bins[:-1], lam_bins[1:]):
    n = np.sum((lam_sel >= lo) & (lam_sel < hi))
    print(lo, hi, "DES Y1 cluster_number", n)


20.003485 234.50368 6729
20 30 DES Y1 cluster_number 762
30 45 DES Y1 cluster_number 376
45 60 DES Y1 cluster_number 123
60 inf DES Y1 cluster_number 91


## (2) Calculate effective area based on footprint mask

In [7]:
from utils.calc_effective_area_DESY1 import (
    load_y1_redmapper_zmask,
    ZMaskAreaCalculator,
    volume_weighted_mean_area_DESY1,
)

zmask_file = DESY3_loc + "des_y1/redmapper_y1a1_public_v6.4_zmask.fits.gz"
hpix, zmax, fracgood = load_y1_redmapper_zmask(zmask_file)

# apply fracgood_min here
m = (fracgood >= 0.8)

# weighted (uses fracgood as weights)
calc = ZMaskAreaCalculator(zmax[m], fracgood[m], nside=4096)

A_volw = volume_weighted_mean_area_DESY1(calc, 0.2, 0.65, return_volume=True)
print("Volume-weighted <A> over [0.2,0.65] =", A_volw["A_volw_deg2"], "deg^2", A_volw["V_eff_total"], "(Mpc/h)^3")

A_volw = volume_weighted_mean_area_DESY1(calc, 0.2, 0.35, return_volume=True)
print("Volume-weighted <A> over [0.2,0.35] =", A_volw["A_volw_deg2"], "deg^2", A_volw["V_eff_total"], "(Mpc/h)^3")




Volume-weighted <A> over [0.2,0.65] = 1436.6893854346458 deg^2 623888358.6268659 (Mpc/h)^3
Volume-weighted <A> over [0.2,0.35] = 1557.8232877356809 deg^2 111074305.50824429 (Mpc/h)^3
